# 🧪 Python Unit Test Generator with GPT, Claude, and Gemini

---

## 🧠 Overview

This notebook demonstrates an AI-powered tool that **automatically generates unit tests** for Python functions using Large Language Models (LLMs).

You can:
- Paste your custom Python function
- Choose from **GPT (OpenAI)**, **Claude (Anthropic)**, or **Gemini (Google)**
- Instantly generate a `unittest`-based test suite for the given function

This project is ideal for:
- Practicing **Test-Driven Development (TDD)**
- Teaching AI-assisted programming
- Automating Python test coverage

---

## ⚙️ How It Works

1. User pastes a Python function (e.g., `def calculate_pi(n): ...`)
2. Selects an LLM model from the dropdown
3. Clicks “Generate Unit Test”
4. The assistant returns Python test code using the `unittest` framework

Generated tests cover:
- Expected outputs for various inputs
- Edge cases (e.g., zero or negative arguments)
- Exception handling

---

## 📦 Requirements

Before running:
- Set up a `.env` file with your API keys:
  ```env
  OPENAI_API_KEY=...
  ANTHROPIC_API_KEY=...
  GOOGLE_API_KEY=...


In [ ]:
# 🧱 1. Import required libraries
import os
from dotenv import load_dotenv
from openai import OpenAI
from anthropic import Anthropic
import google.generativeai as genai
import gradio as gr

In [ ]:
# 🔑 2. Load environment variables
load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
claude_api_key = os.getenv("ANTHROPIC_API_KEY")
gemini_api_key = os.getenv("GOOGLE_API_KEY")


In [ ]:
# 🌐 3. Initialize API clients
openai = OpenAI(api_key=openai_api_key)
claude = Anthropic(api_key=claude_api_key)
genai.configure(api_key=gemini_api_key)


In [ ]:
# 🧠 4. Shared system prompt (for Python unit test generation)
system_prompt = (
    "You are an assistant that writes Python unit tests. "
    "Given a Python function, generate a test class using the 'unittest' framework. "
    "Do not modify the original function. Assume the function is in 'main.py' and properly imported."
)


In [4]:
def chat(user_code, model_type):
    if model_type == "GPT":
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_code}
        ]
        response = openai.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages
        )
        return response.choices[0].message.content

    
    elif model_type == "Claude":
        response = claude.messages.create(
            model="claude-3-haiku-20240307",
            system=system_prompt,
            max_tokens=1024,  # zorunlu
            messages=[{"role": "user", "content": user_code}]
        )
        return response.content[0].text
    
    elif model_type == "Gemini":
        model = genai.GenerativeModel("gemini-1.5-flash")
        chat = model.start_chat(history=[])
        prompt = f"{system_prompt}\n\n{user_code}"
        response = chat.send_message(prompt)
        return response.text


In [5]:
with gr.Blocks() as ui:
    gr.Markdown("## 🧪 Python Unit Test Generator")
    
    with gr.Row():
        code_input = gr.Textbox(
            label="Enter your Python function", 
            lines=12, 
            placeholder="def calculate_pi(n=100): ..."
        )
        model_dropdown = gr.Dropdown(
            choices=["GPT", "Claude", "Gemini"],
            value="GPT",
            label="Select model"
        )
    
    generate_btn = gr.Button("Generate Unit Test")
    code_output = gr.Code(label="Generated Test Code", language="python")

    generate_btn.click(fn=chat, inputs=[code_input, model_dropdown], outputs=code_output)

ui.launch()


* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
